# A Data Collection

In [1]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter 
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import csv

from pandas import ExcelWriter
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

consumer_key="uGTutLYLLbotLOHRNd20ui5MJ"
consumer_secret="8wNj6aX5VATOUpF3JhZLAMHd170hA8eaVIwyq6LfTrlN4J8AAB"
access_token="18916501-vTb85pa5kHneJSA27G4exrrcHvtPScAb6ESDGf2bG"
access_token_secret="LzZ2lPfa583tEbwT4kNYV6EtJUlrGTIOSDdFLOC1xTwvz"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#SamsungFile = open('galaxy.csv', 'w',newline='')
#csvWriter = csv.writer(SamsungFile)
tweet_array = []

twt = tweepy.Cursor(auth_api.search,q="GalaxyS OR Galaxy S",  
                           lang="en" ,
                           since="2018-01-01")

for status in twt.items(20000):
    created_at = status.created_at
    coordinates = status.coordinates
    user_location = status.user.location.encode('utf-8')
    tweet_text = status.text.encode('utf-8')
    fav_count = status.favorite_count
    retweet_count = status.retweet_count
    user_id = status.user.id
    user_name = status.user.name.encode('utf-8')
    user_utc_offset = status.user.utc_offset
    followers_count = status.user.followers_count
    friends_count = status.user.friends_count
        
    full_tweet = [created_at, coordinates, user_location, tweet_text, fav_count, 
                 retweet_count, user_id, user_name, user_utc_offset, followers_count,
                 friends_count]
    tweet_array.append(full_tweet)
    
#    print(full_tweet)
with open('galaxys10.csv', 'w',newline='') as SamsungFile:
    csvWriter = csv.writer(SamsungFile)
    csvWriter.writerow(['created_at','coordinates','user_location','tweet_text','fav_count', 
                 'retweet_count','user_id','user_name','user_utc_offset','followers_count',
                 'friends_count'])
    csvWriter.writerows(tweet_array)
#    csvWriter.writerow([created_at, coordinates, user_location, tweet_text, fav_count, 
#                 retweet_count, user_id, user_name, user_utc_offset, followers_count,
#                 friends_count])

print('done loading')

get_file = 'galaxys10.csv'
clean_file = 'galaxys10_clean.csv'
23

df =pd.read_csv(get_file, sep="\t")
df.drop_duplicates(subset=None, inplace=True)
df.to_csv(clean_file, sep="\t")

print('done cleaning')
  

Rate limit reached. Sleeping for: 854
Rate limit reached. Sleeping for: 859


done loading
done cleaning


In [36]:
df1 = pd.DataFrame(tweet_array)
df1.columns = ['created_at','coordinates','user_location','tweet_text','fav_count', 
                 'retweet_count','user_id','user_name','user_utc_offset','followers_count',
                 'friends_count']

df1.head()

,created_at,coordinates,user_location,tweet_text,fav_count,retweet_count,user_id,user_name,user_utc_offset,followers_count,friends_count
0,2018-11-28 03:51:49,None,b'',"b""RT @ShadowWolfCat: I've been hearing rumors ...",0,3,1010296949352943616,b'\xf0\x9f\x90\xb0Ghostly\xf0\x9f\x91\xbb',None,248,85
1,2018-11-28 03:50:28,None,"b'Toronto, Ontario'",b'RT @MobileSyrup: Samsung Galaxy S10+ specs l...,0,1,877989182,b'Ibrahim Khan',None,147,518
2,2018-11-28 03:49:01,None,b'',b'I liked a @YouTube video https://t.co/AUZkJK...,0,0,2223320683,b'daniel odame brown',None,55,110
3,2018-11-28 03:47:05,None,b'',"b'RT @UniverseIce: Prior to the Galaxy S10, al...",0,81,441669050,b'rizwan',None,134,510
4,2018-11-28 03:46:57,None,"b'Dhaka, Bangladesh'",b'I liked a @YouTube video https://t.co/5JxWui...,0,0,1050007719585832960,b'Hridoy',None,4,141


# Data Cleansing
de-duping

# Data Pre-Processing
Removing Stop Words

In [45]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

df_tweets = df1['tweet_text']
df_tweets = df_tweets.replace('\s+', ' ', regex=True)
df_tweets = df_tweets.replace('\.+', '.', regex=True)
special_characters_list = [':', ';', '?', '}', ')', '{', '(']

for special_char in special_characters_list:
    df_tweets=df_tweets.replace(special_char, '')

df_tweets = df_tweets.replace('((www\.[\s]+)|(https?://[^\s]+))','URL',regex=True)
df_tweets =df_tweets.replace(r'#([^\s]+)', r'\1', regex=True)
df_tweets =df_tweets.replace('\'"',regex=True)

#print(df_tweets)

lem = WordNetLemmatizer()
stem = PorterStemmer()
stop_words = set(stopwords.words('english'))

df_tweets = df_tweets.to_frame(name=None)

df_tweets_sent=df_tweets.to_string(buf=None, columns=None, col_space=None, header=True, index=True, na_rep='NaN', formatters=None, float_format=None, 
sparsify=False, index_names=True, justify=None, line_width=None, max_rows=None, max_cols=None, show_dimensions=False)

fg=stem.stem(df_tweets_sent)

tokens = word_tokenize(fg)
tokens = [word for word in tokens if word not in stop_words]

print(df1.head())
print (tokens)

tweets_array_2 = []
for item in df1:
    tweets_array_2.append(tokens)

df1.head()
    


           created_at coordinates         user_location  \
0 2018-11-28 03:51:49        None                   b''   
1 2018-11-28 03:50:28        None   b'Toronto, Ontario'   
2 2018-11-28 03:49:01        None                   b''   
3 2018-11-28 03:47:05        None                   b''   
4 2018-11-28 03:46:57        None  b'Dhaka, Bangladesh'   

                                          tweet_text  fav_count  \
0  b"RT @ShadowWolfCat: I've been hearing rumors ...          0   
1  b'RT @MobileSyrup: Samsung Galaxy S10+ specs l...          0   
2  b'I liked a @YouTube video https://t.co/AUZkJK...          0   
3  b'RT @UniverseIce: Prior to the Galaxy S10, al...          0   
4  b'I liked a @YouTube video https://t.co/5JxWui...          0   

   retweet_count              user_id  \
0              3  1010296949352943616   
1              1            877989182   
2              0           2223320683   
3             81            441669050   
4              0  1050007719585832960

,created_at,coordinates,user_location,tweet_text,fav_count,retweet_count,user_id,user_name,user_utc_offset,followers_count,friends_count
0,2018-11-28 03:51:49,None,b'',"b""RT @ShadowWolfCat: I've been hearing rumors ...",0,3,1010296949352943616,b'\xf0\x9f\x90\xb0Ghostly\xf0\x9f\x91\xbb',None,248,85
1,2018-11-28 03:50:28,None,"b'Toronto, Ontario'",b'RT @MobileSyrup: Samsung Galaxy S10+ specs l...,0,1,877989182,b'Ibrahim Khan',None,147,518
2,2018-11-28 03:49:01,None,b'',b'I liked a @YouTube video https://t.co/AUZkJK...,0,0,2223320683,b'daniel odame brown',None,55,110
3,2018-11-28 03:47:05,None,b'',"b'RT @UniverseIce: Prior to the Galaxy S10, al...",0,81,441669050,b'rizwan',None,134,510
4,2018-11-28 03:46:57,None,"b'Dhaka, Bangladesh'",b'I liked a @YouTube video https://t.co/5JxWui...,0,0,1050007719585832960,b'Hridoy',None,4,141


In [66]:
from string import punctuation
from collections import Counter

def tokenizer(text):
    
    tokens_leo = [word_tokenize(sent) for sent in sent_tokenize(text)]
    
    tokens_2 = []
    for token_by_sent in tokens_leo:
        tokens_2 +=token_by_sent
        
    tokens_2 = list(filter(lambda t: t.lower() not in stop_words, tokens_2))
    tokens_2 = list(filter(lambda t: t.lower() not in punctuation, tokens_2))
    
    filtered_tokens = []
    for token in tokens_2:
        tokens_2 = wnl.lammatize(tokens_2)
        if re.search('[a-zA-Z]', tokens_2):
            filtered_tokens.append(tokens_2)
            
    filtered_tokens = list(map(lambda tokens_2: token.lower(), filtered_tokens))

    return filtered_tokens  
    

In [67]:
def get_keywords(tokens_2, num):
    return Counter(tokens_2).most_common(num)

In [74]:
def build_article_df(tweets):
    tweet = []
    for index, row in tweets.iterrows():
        try:
            data=row['tweet_text'].strip().replace("'", "")
            data = strip_tags(data)
            soup = BeautifulSoup(data)
            data = soup.get_text()
            data = data.encode('ascii', 'ignore').decode('ascii')
            document = tokenizer(data)
            top_5 = get_keywords(document, 5)
          
            unzipped = zip(*top_5)
            kw= list(unzipped[0])
            kw=",".join(str(x) for x in kw)
            tweet.append((kw, row['user_name'], row['user_id']))
        except Exception as e:
            print (e)
            #print data
            #break
            pass
        #break
    tweet_df = pd.DataFrame(tweet, columns=['keywords', 'user_name', 'user_id'])
    return tweet_df

 row['retweet_count'],row['user_id'],row['user_name'],row['user_utc_offset'],row['followers_count']
                 row['friends_count']
                 , 
                 'retweet_count','user_id','user_name','user_utc_offset','followers_count',
                 'friends_count'

In [75]:
df_csv = pd.read_csv(galaxys10_clean.csv)
data = []
for index, row in df1.iterrows():
    data.append((row['created_at'],row['coordinates'],row['user_location'],row['tweet_text'],row['fav_count']))
data_df = pd.DataFrame(data, columns=['created_at','coordinates','user_location','tweet_text','fav_count'])
    
   

In [76]:
data_df.head()

,created_at,coordinates,user_location,tweet_text,fav_count
0,2018-11-28 03:51:49,None,b'',"b""RT @ShadowWolfCat: I've been hearing rumors ...",0
1,2018-11-28 03:50:28,None,"b'Toronto, Ontario'",b'RT @MobileSyrup: Samsung Galaxy S10+ specs l...,0
2,2018-11-28 03:49:01,None,b'',b'I liked a @YouTube video https://t.co/AUZkJK...,0
3,2018-11-28 03:47:05,None,b'',"b'RT @UniverseIce: Prior to the Galaxy S10, al...",0
4,2018-11-28 03:46:57,None,"b'Dhaka, Bangladesh'",b'I liked a @YouTube video https://t.co/5JxWui...,0


In [78]:
tweet_df = build_article_df(df1)
tweet_df.head()

a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-lik

a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-lik

a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-lik

a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-lik

a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-lik

a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-lik

a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-like object is required, not 'str'
a bytes-lik

,keywords,user_name,user_id
